Script to convert latitudes and longitudes into human readable addresses using google maps

In [125]:
import pandas as pd
import requests
import logging
import time

logger = logging.getLogger("root")
logger.setLevel(logging.DEBUG)

# create console log  handler
ch = logging.StreamHandler()
ch.setLevel(logging.DEBUG)
logger.addHandler(ch)

In [126]:
# declare google maps api key, google maps endpoint and dataset

api_key =""
api_url = "https://maps.googleapis.com/maps/api/geocode/json?latlng="
source_data= pd.read_csv('E:/ACTIVE/Datasets/Site.csv')

In [129]:
# sample of the dataset

coordinates = source_data.head()[["Site Name","LAT","LONG"]]
coordinates

,Site Name,LAT,LONG
0,ABAYITA_ABABIRI,0.0935,32.5009
1,ACHOLI_INN,2.7831,32.2914
2,ARUA_EDIOFE,3.0111,30.8962
3,ARUA_TOWN,3.0238,30.9084
4,ATIAK,3.2579,32.1210


In [130]:
# function to reverse geocode an address

def reverse_geocoding(api_url,lat,long,api_key):
    full_url = api_url+str(lat)+","+str(long)+"&key="+api_key+""
    response = requests.get(full_url)  
    response_json = response.json()
    result =(response_json["results"][0]['formatted_address']).split(',',1)[1]


#check response if results are empty
    if len(response_json) == 0:
        output = {
            "Address" : None,
            "Status":"None"
           }
    else:    
        output = {
            "Address" : result,
            "Status":response_json["status"]
        }
    return list(output.values())      

In [121]:
# process a list of coordinates to get the corresponding addresses

my_list=[]
for index, row in coordinates.iterrows():
            sitename=row["Site Name"]
            latitude= row["LAT"]
            longitude=row["LONG"]
            address = reverse_geocoding(api_url,latitude,longitude,api_key)
            status =address[1]
            if status == "None":
                logger.warning("ErrorReverseGeocoded {}: {}".format(sitename, status))
            
            elif status=="OK":
                logger.debug("ReverseGeocoded: {}: {}".format(sitename,status))
                my_list.append([sitename,latitude,longitude,address[0]])  
print(my_list)